# VacationPy
----


In [1]:
# Importing dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

gmaps.configure(api_key=gkey)

## Read WeatherPy results into dataframe

In [2]:
# Creating path object from WeatherPy output csv
csv_path = os.path.join('..','WeatherPy', 'output_data', 'cities.csv')

# Reading csv into dataframe
vac_df = pd.read_csv(csv_path).reset_index(drop=True).drop(['Unnamed: 0'], axis=1)

# Printing sample of dataframe
vac_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %
0,Severo-Kuril'sk,RU,50.68,156.12,54.01,91,9.48,100
1,Rikitea,PF,-23.12,-134.97,72.82,79,14.25,81
2,Pevek,RU,69.70,170.31,37.42,93,3.51,100
3,Magdalena,PH,14.20,121.43,80.60,88,2.24,95
4,Mataura,NZ,-46.19,168.86,52.05,88,1.81,34


## Humidity Heatmap

In [3]:
# Pulling dataframe columns into variables
locations = vac_df[['Latitude', 'Longitude']]
humidity = vac_df['Humidity %']

In [4]:
# Initializing map
fig = gmaps.figure()

# Creating heatmap layer, coordinates set on locations list, with weighting determined by humidity %
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                              dissipating=False, max_intensity=100, point_radius=2)
# Overlaying heatmap layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Filter cities into new dataframe with my ideal weather conditions

My ideal weather is between 60-75 degrees fahrenheit, very cloud, with no wind

In [5]:
ideal_df = vac_df.loc[(vac_df['Max Temp (F)'] <= 75) & (vac_df['Max Temp (F)'] >= 60) 
                      & (vac_df['Cloudiness %'] > 90) & (vac_df['Wind (mph)'] < 5), :]

ideal_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Wind (mph),Cloudiness %
20,Linjiang,CN,41.81,126.91,70.18,88,2.84,93
64,Bousso,TD,10.48,16.71,71.58,95,1.21,100
126,Margate,GB,51.38,1.39,64.00,71,1.99,100
128,Lashio,MM,22.93,97.75,70.39,100,1.32,100
154,Rio Grande,BR,-32.03,-52.10,62.69,90,3.22,97
155,Drezdenko,PL,52.84,15.83,64.99,93,1.99,100
209,Muros,ES,42.78,-9.06,64.00,78,4.00,100
230,Lucapa,AO,-8.42,20.74,68.94,42,4.16,92
247,Medak,IN,18.03,78.27,75.00,65,4.00,100
250,Butuan,PH,8.95,125.54,74.97,91,2.48,100


## Creating a map overlay of hotels in my preferred weather conditions

In [6]:
# Assigning Maps Places syntax as a base call variable
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Creating empty hotels list to store Places call results
hotels = []

# Iterating through dataframe and extracting variables to populate call parameters
for index, row in ideal_df.iterrows():
    params = {"location": f"{row['Latitude']}, {row['Longitude']}",
              "radius": 5000,
              "type": 'hotel',
              "key": gkey}
    
    # Error handling for locations without multiple results
    try:
        response = requests.get(base_url, params=params).json()
        hotels.append(response['results'][1]['name'])
    except:
        hotels.append('None')

In [7]:
# Cleaning up hotel name format
hotels = [i.lower() for i in hotels]
hotels = [i.title() for i in hotels]
ideal_df['Hotel Name'] = hotels

# Renaming dataframe variable
hotel_df = ideal_df

/Users/troyramsey/opt/anaconda3/envs/UODataClass/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
# Setting up template for info boxes on marker layer
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Populating list with hotel information
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Creating and formatting marker layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='black',
    stroke_color='black', scale=2,
    info_box_content=hotel_info
)

# Overlaying marker layer onto heatmap
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))